### library setttings

In [1]:
# others
import pandas as pd
import numpy as np
import argparse
import datetime
from copy import deepcopy # Add Deepcopy for args
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)


# read file
file_path = './data/merged_data.h5'
data = pd.read_hdf(file_path)


1.8.1
Populating the interactive namespace from numpy and matplotlib


### 데이터 확인

In [2]:
data.head()

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,9,0.983614,0.983614,0.983128,0.983246,0.001334,10.650987,0.009855,0.000848,6.771755
1,0,1,9,0.983245,0.983612,0.982453,0.982693,0.001425,11.375689,0.016137,0.000697,5.565188
2,0,2,9,0.982694,0.983612,0.982403,0.983002,0.001542,12.301942,0.014166,0.000905,7.225459
3,0,3,9,0.983009,0.984848,0.983009,0.984486,0.002520,20.134695,0.021557,0.001171,9.353000
4,0,4,9,0.984233,0.984606,0.983612,0.984164,0.002818,22.515448,0.021434,0.001799,14.372534


### 데이터 정규화 및 스플릿 

In [4]:
def datascaling_and_spliting(dataframe):
    '''  scaling version2 : 거래량 데이터외의 데이터도 넣을 경우 '''
    
    n = data.sample_id.nunique()

    # train, val, test = 7 : 2  : 1
    train_df = data[data.sample_id < int(n*0.7)]
    val_df =data[(data.sample_id >= int(n*0.7)) & (data.sample_id < int(n*0.9))]
    test_df = data[data.sample_id >= int(n*0.9)]
    
    # create scaler and dafaframes
    scaler = MinMaxScaler()

    # scaler fitting
    scaler = scaler.fit(train_df.iloc[:, -5:])
    
    # transforms each dataframes
    train_df.iloc[:, -5:] = scaler.transform(train_df.iloc[:, -5:])
    val_df.iloc[:, -5:] = scaler.transform(val_df.iloc[:, -5:])
    test_df.iloc[:, -5:] = scaler.transform(test_df.iloc[:, -5:])

    return train_df, val_df, test_df, scaler

In [5]:
scaled_train_df, scaled_val_df, scaled_test_df, scaler = datascaling_and_spliting(data)

C:\Anaconda3\envs\venv\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Anaconda3\envs\venv\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Anaconda3\envs\venv\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [8]:
# 정규화 확인
scaled_test_df.head()

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
1630500,1087,0,9,1.010196,1.011977,1.008776,1.009570,0.019519,0.023766,0.040007,0.011853,0.012426
1630501,1087,1,9,1.009476,1.010790,1.007363,1.009166,0.015788,0.019197,0.045144,0.011924,0.012481
1630502,1087,2,9,1.009166,1.009839,1.007852,1.009602,0.015113,0.018377,0.041034,0.013681,0.014323
1630503,1087,3,9,1.009602,1.009879,1.007869,1.009652,0.021918,0.026666,0.039429,0.023879,0.025008
1630504,1087,4,9,1.009614,1.011977,1.009363,1.011740,0.013460,0.016386,0.035897,0.014611,0.015312


### 데이터 generator 생성

In [10]:
class DatasetGenerater(Dataset):
    ''' 설명 생략 '''
    
    def __init__(self, dataframe, x_frames, y_frames, status_idx = None):

        self.grouped_dataframe = dataframe.groupby('sample_id')
        self.sample_length = dataframe.sample_id.nunique()
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        if status_idx == 'train':
            self.status_idx = 0
            print(self.status_idx)
            
        elif status_idx == 'validation':
            self.status_idx = 845
            print(self.status_idx)
            
        else:
            self.status_idx = 1087
            print(self.status_idx)
        
        
        
    def __len__(self):
        return self.sample_length
    
    def __getitem__(self, idx):
        
        idx += self.status_idx
        
        one_sample_df = self.grouped_dataframe.get_group(idx)
        
        one_sample_df = one_sample_df[-(self.x_frames + self.y_frames):]
        one_sample_arr = one_sample_df.drop(['sample_id','time', 'coin_index'], axis = 1).values

        # get values
        X = one_sample_arr[:self.x_frames, :]
        y = one_sample_arr[self.x_frames:, :]
        
        return X, y

### hpyperparameter setting

In [11]:

# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

seed = 777
np.random.seed(seed)
torch.manual_seed(seed)


# ====== Model Capacity options ===== #
args.input_dim = 9
args.hidden_dim = 50
args.output_dim = 1
args.num_layers = 3
args.batch_size = 24
args.dropout = 0.2
args.target_len = 120
args.use_bn = True

# ====== Dataset Generating options ====== #
args.x_frames = 255
args.y_frames = 120

# ====== Model training options ===== #
args.num_epoch = 30
args.learning_rate = 0.001
args.L2_rate = 0.0001
args.training_prediction = 'mixed_teacher_forcing'
args.teacher_forcing_ratio = 0.6


# dataset partitioning
trainset = DatasetGenerater(scaled_train_df, x_frames = args.x_frames, y_frames = args.y_frames, status_idx = 'train')
valset = DatasetGenerater(scaled_val_df, x_frames = args.x_frames, y_frames = args.y_frames, status_idx = 'validation')
testset = DatasetGenerater(scaled_test_df, x_frames = args.x_frames, y_frames = args.y_frames, status_idx = 'test')
partition = {'train': trainset, 'val':valset, 'test':testset}

device is cpu
0
845
1087


In [71]:
scaled_train_df[1378:]

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
1378,0,1378,9,0.999629,1.000116,0.999143,1.000000,0.010375,0.012053,0.011322,0.010341,0.010341
1379,0,1379,9,1.000000,1.000123,0.999388,0.999998,0.003003,0.003489,0.007992,0.003344,0.003345
1380,0,1380,9,0.999389,0.999630,0.999261,0.999385,0.002810,0.003264,0.006793,0.002225,0.002224
1381,0,1381,9,0.999593,1.000000,0.999265,0.999266,0.009934,0.011539,0.011122,0.011090,0.011088
1382,0,1382,9,0.999266,0.999618,0.999262,0.999322,0.009332,0.010837,0.008991,0.010919,0.010914
...,...,...,...,...,...,...,...,...,...,...,...,...
1267495,844,1495,9,1.002049,1.003136,1.001533,1.002579,0.006706,0.022019,0.037540,0.004785,0.013523
1267496,844,1496,9,1.002579,1.003686,1.001733,1.003645,0.004476,0.014699,0.019656,0.004404,0.012451
1267497,844,1497,9,1.003636,1.003919,1.001591,1.001806,0.003378,0.011099,0.021097,0.002601,0.007354
1267498,844,1498,9,1.001806,1.001881,0.998480,0.999429,0.014149,0.046351,0.053038,0.007034,0.019832


In [60]:
trainloader = DataLoader(partition['train'], batch_size = args.batch_size, shuffle = True, drop_last = True)

for i, (X,y) in enumerate(trainloader):
    print(X.shape, y.shape)
    
    break

torch.Size([24, 255, 9]) torch.Size([24, 120, 9])


In [42]:
class Encoder(nn.Module):
    ''' Encodes time-serise sequence '''
    
    def __init__(self, input_dim, hidden_dim, num_layers):
        
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are 2 stacked LSTMs)
        '''
        
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        
        # define LSTM layer
        self.rnn = nn.GRU(self.input_dim, self.hidden_dim, self.num_layers)
    
    def forward(self, X):
        
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        
        rnn_out, self.hidden = self.rnn(X)
        return rnn_out, self.hidden
    
    def init_hidden(self, batch_size):
        
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

    
class Decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, output_dim, hidden_dim, num_layers, dropout, use_bn):
        
        '''
        : param output_dim:     the number of features in the input X
        : param hidden_dim:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(Decoder, self).__init__()
        self.output_dim = output_dim 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.use_bn = use_bn
        
        self.rnn = nn.GRU(self.output_dim, self.hidden_dim, self.num_layers)
        self.fc_out = self.regressor()
        
    def forward(self, x_input, encoder_hidden_states):
               
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        
        # print(f'decoder input size {x_input.shape} to {x_input.reshape(1, -1, self.output_dim).shape}')
        rnn_out, self.hidden = self.rnn(x_input.reshape(1, -1, self.output_dim), encoder_hidden_states)
        output = self.fc_out(rnn_out.squeeze(0)) 
        
        return output, self.hidden
    
    def regressor(self):

        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)

        return regressor

class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder, device, target_len, training_prediction, teacher_forcing_ratio):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.target_len = target_len
        self.training_prediction = training_prediction
        self.teacher_forcing_ratio = teacher_forcing_ratio
        
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    
    def forward(self, X, y):
            
            if y is None:
                self.training_prediction = 'recursive'
                
                
            # initialize
            outputs = torch.zeros(self.target_len, X.shape[1], self.decoder.output_dim).to(self.device)
            
            encoder_hidden = self.encoder.init_hidden(X.shape[1])
        
            # encoder process
            encoder_output, encoder_hidden = self.encoder(X)  
            
            #decoder process
            ''' last X sequence, shape = [batch_size, open_index = 0] '''    
            decoder_input = X[-1, :, 0]  
            decoder_hidden = encoder_hidden  
            

            # ======================================================================================#
            if self.training_prediction == 'recursive':
                # predict recursively
                for t in range(self.target_len): 
                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                    
                    outputs[t] = decoder_output 
                    decoder_input = decoder_output
                    
           # ======================================================================================#
            if self.training_prediction == 'teacher_forcing':
                # use teacher forcing
                if random.random() < self.teacher_forcing_ratio:
                    for t in range(self.target_len): 
                        decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                        outputs[t] = decoder_output
                        decoder_input = y[t, :].unsqueeze(1)
                        
                # predict recursively 
                else:
                    for t in range(self.target_len): 
                        decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                        outputs[t] = decoder_output
                        decoder_input = decoder_output
                        
            # ======================================================================================#
            if self.training_prediction == 'mixed_teacher_forcing':
                # predict using mixed teacher forcing
                for t in range(self.target_len):

                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                    outputs[t] = decoder_output 
                                    
                    # predict with teacher forcing
                    if random.random() < self.teacher_forcing_ratio:
                        decoder_input = y[t, :].unsqueeze(1)

                    # predict recursively 
                    else:
                        decoder_input = decoder_output

            # ======================================================================================#      
            return outputs




In [47]:

trainloader = DataLoader(partition['train'], batch_size = args.batch_size, shuffle = False, drop_last = True)

# Encoder
enc = Encoder(args.input_dim, args.hidden_dim, args.num_layers)

# Decoder
dec = Decoder(args.output_dim, args.hidden_dim, args.num_layers, args.dropout, args.use_bn)

# Seq2Seq model
model = Seq2Seq(enc, dec, args.device, args.target_len, args.training_prediction, args.teacher_forcing_ratio)
model.to(args.device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)

# model's mode setting
model.train()

# model initialization
model.zero_grad()
train_loss = 0.0

# train
for i, (X, y) in enumerate(trainloader):

    X = X.transpose(0, 1).float().to(args.device)

    # y = y[:,:, 0].transpose(0, 1).float().to(args.device)
    y_true = y[:,:, 0].transpose(0, 1).float().to(args.device)
    
    # zero the gradient
    optimizer.zero_grad()

    # en-decoder outputs tensor 
    y_pred = model(X, y_true)
    
    
    # compute the loss 
    loss = loss_fn(y_true.unsqueeze(2), y_pred)

    #backpropagation
    loss.backward()
    optimizer.step()

    # get the batch loss
    train_loss += loss.item()

train_loss = train_loss / len(trainloader)

torch.Size([120, 24]) torch.Size([120, 24, 1])
torch.Size([120, 24]) torch.Size([120, 24, 1])
torch.Size([120, 24]) torch.Size([120, 24, 1])
torch.Size([120, 24]) torch.Size([120, 24, 1])


KeyboardInterrupt: 

In [53]:

def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
    
   
    # data load
    trainloader = DataLoader(partition['train'], batch_size = args.batch_size, shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()

    # model initialization
    model.zero_grad()
    train_loss = 0.0
    
    # train
    for i, (X, y) in enumerate(trainloader):
        
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:,:, 0].transpose(0, 1).float().to(args.device)

        # zero the gradient
        optimizer.zero_grad()
        
        # en-decoder outputs tensor 
        y_pred = model(X, y_true)

        # compute the loss 
        loss = loss_fn(y_true, y_pred.squeeze(2))
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()
        
    train_loss = train_loss / len(trainloader)
    return model, train_loss


def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], batch_size = args.batch_size, shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):
            
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, 0].transpose(0, 1).float().to(args.device)

            # en-decoder outputs tensor 
            y_pred = model(X, None)
            
            # compute the loss 
            loss = loss_fn(y_true, y_pred.squeeze(2))

            # get the batch loss
            val_loss += loss.item()
            
    val_loss = val_loss / len(valloader)
 
    return val_loss


def test(model, partition, scaler, args):
    ''' model test '''
    
    # data load
    testloader = DataLoader(partition['test'], batch_size = 1, shuffle = False, drop_last = False)
    
    # model's mode setting
    model.eval()
    
    test_mae = 0.0
    score_list = list()
    item_loss_list = list()
    
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            
            X = X.transpose(0, 1).float().to(args.device)            
            y_true = y[:, :, 0].transpose(0, 1).float().to(args.device)
            
            # en-decoder outputs tensor 
            y_pred = model(X, None)
            y_pred = y_pred.squeeze(2)
            
            # y values to cpu
            y_true = y_true.cpu().detach().numpy()
            y_pred = y_pred.cpu().detach().numpy()

            # get the batch loss
            test_mae += mean_absolute_error(y_true, y_pred)
#             score = r2_score(y_pred = y_pred.transpose(), y_true = y_true.transpose(), multioutput = 'uniform_average')
#             score_list.append(score)
            item_loss_list.append(abs(y_true - y_pred))
            
            
    test_mae /= len(testloader)
#     score /= len(testloader)
    
    return test_mae, item_loss_list



In [57]:
def experiment(partition, scaler, args):

    # Encoder
    enc = Encoder(args.input_dim, args.hidden_dim, args.num_layers)
    
    # Decoder
    dec = Decoder(args.output_dim, args.hidden_dim, args.num_layers, args.dropout, args.use_bn)
    
    # Seq2Seq model
    model = Seq2Seq(enc, dec, args.device, args.target_len, args.training_prediction, args.teacher_forcing_ratio)
    model.to(args.device)
    
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []

    for epoch in range(args.num_epoch):
        
        start_time = time.time()

        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss = validate(model, partition, loss_fn, args)
        
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print('Epoch {},Loss(train/val) {:.3f}/{:.3f}. Took {:.2f} sec'.format(epoch+1, train_loss, val_loss, end_time-start_time))
    
    # test part
    # test_mae, item_loss_list = test(model, partition, scaler, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses #epoch 수에 의존
    result['val_losses'] = val_losses 
    
    #result['test_mae'] = test_mae.round(3).item()
    
    # result['r2'] = np.array(score_list).mean().round(3)
    # item_loss = np.array(item_loss_list).mean(axis=0).mean(axis=0).astype(int)
    # item_loss = list([int(x) for x in item_loss])
    # result['item_loss'] = item_loss
     
    return vars(args), result, model

In [56]:
experiment(partition, scaler, args)

Epoch 1,Loss(train/val) 0.692/71.133. Took 36.70 sec


KeyboardInterrupt: 

In [ ]:
print(
    f'''
    {df.high.max()}
    {df.low.max()}
    {df.open.max()}
    {df.close.max()}
    
    
    {df.high.min()}
    {df.low.min()}
    {df.open.min()}
    {df.close.min()}
    
    '''
    
    ''' high - low = 변동폭 \n'''
    ''' 음봉양봉 구분 추가 가능'''
)
